# 🚗 License Plate Detection & OCR (YOLOv8 + Tesseract OCR)
This Colab notebook demonstrates how to detect and recognize license plates from video using **YOLOv8**, **OpenCV**, and **Tesseract OCR**.

In [ ]:
!pip install ultralytics opencv-python pytesseract
!apt install tesseract-ocr -y

In [ ]:
import cv2
import pytesseract
from ultralytics import YOLO
import os
import re
import pandas as pd
from google.colab import files
from IPython.display import display, clear_output
from PIL import Image
import matplotlib.pyplot as plt


## 📥 Upload a Video File

In [ ]:
uploaded = files.upload()
video_path = list(uploaded.keys())[0]


## 🧠 Load YOLOv8 Model

In [ ]:
# Replace with your custom model if available
model = YOLO("yolov8n.pt")  # or use 'best.pt'


In [ ]:
def preprocess_for_ocr(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, None, fx=3, fy=3, interpolation=cv2.INTER_LINEAR)
    blur = cv2.GaussianBlur(resized, (5, 5), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh


## 🔁 Run Detection and OCR on Each Frame

In [ ]:
cap = cv2.VideoCapture(video_path)
frame_count = 0
ocr_results = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    results = model.predict(frame, imgsz=640, conf=0.5, device='cpu')[0]

    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        cropped = frame[y1:y2, x1:x2]
        preprocessed = preprocess_for_ocr(cropped)
        text = pytesseract.image_to_string(preprocessed, config="--psm 6")
        cleaned = re.sub(r'[^A-Z0-9\-]', '', text.upper())

        if 2 < len(cleaned) <= 12:
            ocr_results.append((frame_count, cleaned))
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(frame, cleaned, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

    clear_output(wait=True)
    display(Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))

cap.release()


## 💾 Save OCR Results to CSV

In [ ]:
df = pd.DataFrame(ocr_results, columns=["Frame", "License_Plate_Text"])
df.to_csv("video_ocr_results.csv", index=False)
files.download("video_ocr_results.csv")


---
🚀 Notebook generated on: **2025-07-15**